# Transformation examples

## Introduction
This notebook illustrates some tranformation capabilities provided by plaid-ops. 

In [ ]:
import os

from datasets import load_dataset
from PIL import Image as PILImage
from IPython.display import Image as IPyImage, display
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)

from plaid_ops.common.visualization import plot_sample_field
from plaid_ops.mesh.feature_engineering import update_dataset_with_sdf
from plaid_ops.mesh.transformations import (
    compute_bounding_box,
    project_on_other_datset,
    project_on_regular_grid,
)

hf_dataset = load_dataset(
    "PLAID-datasets/2D_Multiscale_Hyperelasticity", split="all_samples"
)
pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("DOE_train")[:2]
dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=2)

In [ ]:
dataset = update_dataset_with_sdf(dataset)

img_name = "transformation_1.png"
if os.environ.get("READTHEDOCS") == "True":
    display(IPyImage(filename=img_name))
else:
    img_array = plot_sample_field(
        dataset[ids[0]],
        "sdf",
        title="Unstructured mesh",
        show_edges=True,
        scalar_bar_args={"title": "sdf"},
    )
    img = PILImage.fromarray(img_array)
    img.save(img_name)
    display(img)

In [ ]:
bbox = compute_bounding_box(dataset)
projected_dataset = project_on_regular_grid(
    dataset, dimensions=[101, 101], bbox=bbox, verbose=True
)

img_name = "transformation_2.png"
if os.environ.get("READTHEDOCS") == "True":
    display(IPyImage(filename=img_name))
else:
    img_array = img = plot_sample_field(
        projected_dataset[ids[0]],
        "sdf",
        title="Projection on regular grid mesh",
        show_edges=True,
        scalar_bar_args={"title": "sdf"},
    )
    img = PILImage.fromarray(img_array)
    img.save(img_name)
    display(img)

In [ ]:
inv_projected_dataset = project_on_other_datset(
    projected_dataset, dataset, verbose=True
)

img_name = "transformation_3.png"
if os.environ.get("READTHEDOCS") == "True":
    display(IPyImage(filename=img_name))
else:
    img_array = img = plot_sample_field(
        inv_projected_dataset[ids[0]],
        "sdf",
        title="Projection back to inital mesh",
        show_edges=True,
        scalar_bar_args={"title": "sdf"},
    )
    img = PILImage.fromarray(img_array)
    img.save(img_name)
    display(img)